In [36]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import Dataset
from transformers import DataCollatorForLanguageModeling, TrainingArguments
from trl import SFTTrainer
import pandas as pd

In [37]:
model_id = "google/gemma-2-2b"
data_dir = "./qna_data"
output_dir = './results'
token = os.environ.get('HUGGINGFACE_TOKEN')

In [38]:
# 모델 및 토크나이저 로드
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [39]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map={"":0},
    token=token
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [40]:
# 데이터셋 로드 및 전처리
def load_qna_files(data_dir):
    files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]
    data = []
    for file in files:
        dataset = pd.read_csv(file)
        print(f'Sample data of {file}')
        print(dataset.head(5))
        for index, row in dataset.iterrows():
            data.append(f"Question: {row['Question']}\nAnswer: {row['Answer']}")
    return data

data = load_qna_files(data_dir)
print(f'Sample of refined data')
print(data[:30])

Sample data of ./qna_data/lecture1_qna.csv
                               Question  \
0           What is the focus of CS230?   
1  What teaching format does CS230 use?   
2  Who are the co-instructors of CS230?   
3          What is a flipped classroom?   
4     What is the role of TAs in CS230?   

                                              Answer  
0  CS230 focuses on deep learning and practical a...  
1  CS230 uses a flipped classroom format with onl...  
2  Andrew Ng and Kian Katanforosh are the co-inst...  
3  Students watch videos at home, leaving class t...  
4  TAs provide guidance, mentorship, and project ...  
Sample data of ./qna_data/lecture2_qna.csv
                               Question  \
0      What is deep learning intuition?   
1    What are the two parts of a model?   
2  What is a logistic regression model?   
3              What does CNN stand for?   
4              What is a loss function?   

                                              Answer  
0  A way to

In [41]:
def preprocess_data(texts, tokenizer, max_length=512):
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length, return_tensors="pt")
    return encodings

# 전처리된 데이터셋
encodings = preprocess_data(data, tokenizer)

In [42]:
# Hugging Face Dataset 객체로 변환
dataset = Dataset.from_dict({"input_ids": encodings["input_ids"], "attention_mask": encodings["attention_mask"]})

# 데이터셋 샘플 출력
print(f"Dataset size: {len(dataset)}")
print("Sample dataset entry:", dataset[0])

Dataset size: 290
Sample dataset entry: {'input_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [43]:
# 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [44]:
# Fine-tuning을 여러 번 가능하게 하는 함수
def fine_tune_model(model, dataset, tokenizer, output_dir="./results", epochs=3):
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=epochs,
        per_device_train_batch_size=1,
        save_steps=500,
        save_total_limit=2,
        logging_steps=100,
        learning_rate=5e-5,
        fp16=True,  # GPU의 성능을 최대한 활용하기 위한 설정
    )
    
    trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        tokenizer=tokenizer,
        data_collator=data_collator
    )
    
    trainer.train()
    
    # 모델 저장
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Model saved to {output_dir}")

# Fine-tuning 실행
fine_tune_model(model, dataset, tokenizer, output_dir)

/home/work/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/work/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/home/work/.local/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
100,3.232900
200,3.162200
300,2.758900
400,1.531700
500,1.454200
600,1.250000
700,0.685000
800,0.642900


Model saved to ./results


In [45]:
def query_model(model, tokenizer, prompt, max_length=512):
    device = "cuda:0"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [46]:
question = "When do we use CNN?"
sample_prompt = f"Question: {question}\nAnswer:"
response = query_model(model, tokenizer, sample_prompt)
print(f"Answer: {response}")

RuntimeError: Index put requires the source and destination dtypes match, got Float for the destination and Half for the source.